# Unet Satcen Finetuning

Fine tune the Unet model trained on the Airbus dataset using the Satcen dataset.

## Construct the Training Dataset

Load and split the Satcen dataset into train and test sets.

- train: 50% 
- validation: 25%
- test: 25%

In [70]:
import os
import pathlib
from PIL import Image, ImageDraw as Drawer
import matplotlib.pyplot as plt
import numpy as np
import ijson
from sklearn.model_selection import train_test_split
import shutil

# path to satcen dataset
satcen_path = pathlib.Path('../satcen_dataset/').resolve()

In [49]:
# load the Satcen image filenames
satcen = [f for f in os.listdir(satcen_path/'pictures')]
print('Total number of Satcen images:', len(satcen))

Total number of Satcen images: 2170


### Generate Ground Truth Labels

Compute ground truth for images in the Satcen dataset based on the given bounding boxes.
Ground truth labels should be saved as *.png* files in the *satcen_dataset/labeled_images_binary* folder. Files should have _L at the end of the filename.

In each ground truth image:
- 0 pixel value represents background pixel
- 1 pixel value represents ship pixel

In [50]:
# compute segmentation mask of a Satcen image based on its bounding box
# @param img_name - filename of the image for which to compute ground truth
# @param bounding_boxes - list of bounding boxes for that image
# @param save - whether or not to save the mask
# @param path - where to save the mask
# @return mask - PIL image representing the mask of the given image
def get_image_mask_satcen(img_name, bounding_boxes, path, save = False):
    
    # create the skeleton of the segmentation mask
    # size is constant for satcen dataset, 256 x 256
    mask = Image.fromarray(np.zeros((256, 256)))

    # get a drawer object
    draw = Drawer.Draw(mask)

    # loop through the bounding boxes
    for bb in bounding_boxes:

        # convert coordinates to required format
        coords = [(c['x'], c['y']) for c in bb]

        # draw the bounding box on the image
        draw.polygon(coords, fill=1)

    if save:
        mask.convert('RGB').save(path/f'{img_name[:-4]}_L.png')

    return mask

# load the json file containing bounding boxes
satcen_json = [entry for entry in ijson.items(open(satcen_path/'SatCen_skiffs256.json'), 'batch.annotations.item')]

# pre-process the json
# resulting format should be
# {name: ..., boundind_boxes: [[{'x': ..., 'y': ...}, ...], ...]}
satcen_json = list(map(lambda x: {'name': x['name'], 'bounding_boxes': [e['data'] for e in x['objects']]}, satcen_json))

# function that generates and saves masks for Satcen images
# @param path - where to save the masks
def save_all_satcen_masks(path):

    # delete all the files in the target directory
    for f in os.listdir(path):
        os.remove(path/f)

    # loop through all satcen images
    for img in satcen:

        # get the corresponding list of bounding boxes
        json_entry = [e for e in satcen_json if e['name'] == img]
        bounding_boxes = json_entry[0]['bounding_boxes'] if len(json_entry) > 0 else []

        # generate and save mask
        get_image_mask_satcen(img, bounding_boxes, path, save=True)

save_all_satcen_masks(satcen_path/'labeled_images_binary')

### Split Dataset

Split the entire Satcen dataset into training (50%), validation (25%) and testing (25%) datasets. Save the training and validation data in the *satcen_dataset/train_valid* folder.

In [60]:
# construct the ground truth labels array
y = []
positives = 0
negatives = 0

# loop through all Satcen images
for filename in satcen:
    mask_arr = np.array(Image.open(satcen_path/'labeled_images_binary'/f'{filename[:-4]}_L.png'))

    if np.any(mask_arr == 1): # positive observation
        y.append(1) 
        positives += 1
    else: # negative observation
        y.append(0)
        negatives += 1

print(f'Number of positive observations: {positives} ({positives / len(satcen) * 100}%)')
print(f'Number of negative observations: {negatives} ({negatives / len(satcen) * 100}%)')

# sanity check that image filenames and corresponding ground truth label 
# are in the correct order in the array
for i in range(len(satcen)):

    img = satcen[i]
    mask = np.array(Image.open(satcen_path/'labeled_images_binary'/f'{img[:-4]}_L.png'))

    if (np.any(mask == 1) and y[i] == 0) or (not np.any(mask == 1) and y[i] == 1):
        raise Exception('Labels do not correspond for ' + img)

Number of positive observations: 1525 (70.27649769585254%)
Number of negative observations: 645 (29.723502304147466%)


In [71]:
# # make the splits
# train_data, validation_test_data = train_test_split(satcen, test_size=0.25, stratify=y)
# vt_y = [y[satcen.index(f)] for f in validation_test_data]
# validation_data, test_data = train_test_split(validation_test_data, test_size=0.5, stratify=vt_y)

# # check how many positive and negative observations are in each subset
# print('### Train Data ###')
# print('Size:', len(train_data))
# train_pos = len([f for f in train_data if y[satcen.index(f)] == 1])
# train_neg = len([f for f in train_data if y[satcen.index(f)] == 0])
# print(f'Positive observations: {train_pos} ({train_pos / len(train_data) * 100}%)')
# print(f'Negative observations: {train_neg} ({train_neg / len(train_data) * 100}%)', '\n')

# print('### Validation Data ###')
# print('Size:', len(validation_data))
# validation_pos = len([f for f in validation_data if y[satcen.index(f)] == 1])
# validation_neg = len([f for f in validation_data if y[satcen.index(f)] == 0])
# print(f'Positive observations: {validation_pos} ({validation_pos / len(validation_data) * 100}%)')
# print(f'Negative observations: {validation_neg} ({validation_neg / len(validation_data) * 100}%)', '\n')

# print('### Test Data ###')
# print('Size:', len(test_data))
# test_pos = len([f for f in test_data if y[satcen.index(f)] == 1])
# test_neg = len([f for f in test_data if y[satcen.index(f)] == 0])
# print(f'Positive observations: {test_pos} ({test_pos / len(test_data) * 100}%)')
# print(f'Negative observations: {test_neg} ({test_neg / len(test_data) * 100}%)', '\n')

# # remove all files currently in the train_valid folder
# for f in os.listdir(satcen_path/'train_valid'):
#     os.remove(satcen_path/'train_valid'/f)

# # save training data
# for f in train_data:
#     src = satcen_path/'pictures'/f
#     dst = satcen_path/'train_valid'/f
#     shutil.copy(src, dst)

# # save validation data
# for f in validation_data:
#     src = satcen_path/'pictures'/f
#     dst = satcen_path/'train_valid'/f
#     shutil.copy(src, dst)

print('Total number of training and validation examples:', len(os.listdir(satcen_path/'train_valid')))

### Train Data ###
Size: 1627
Positive observations: 1143 (70.2519975414874%)
Negative observations: 484 (29.7480024585126%) 

### Validation Data ###
Size: 271
Positive observations: 191 (70.47970479704797%)
Negative observations: 80 (29.520295202952028%) 

### Test Data ###
Size: 272
Positive observations: 191 (70.22058823529412%)
Negative observations: 81 (29.77941176470588%) 

Total number of training and validation examples: 1898
